In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import json
import os
from datasets import Dataset as HFDataset


# Funciones para carga de los datasets

In [ ]:
def load_image_dataset(split_name: str):
  """Carga el split del dataset especificado, cada fila es una ruta con su etiqueta asociada."""
  root_folder = f"./{split_name}"
  image_paths = []
  example_paths = {}
  image_labels = []

  with open(f"{root_folder}/{split_name}.json", 'r') as f:
    labels_dict = json.load(f)
  for root, _, files in os.walk(root_folder):
    for _f in files:
      if _f.endswith(".png"):
          full_path = os.path.join(root, _f)
          rel = os.path.join(*full_path.split(os.sep)[-2:])
          try:
            if labels_dict[rel] not in example_paths.keys():
                example_paths[labels_dict[rel]] = rel
            image_labels.append(labels_dict[rel])
            image_paths.append(full_path)
          except KeyError:
            continue
  dataset = HFDataset.from_dict({
      "image": image_paths,
      "label": image_labels
  })

  dataset = dataset.class_encode_column("label")
  return dataset

# Funciones de dibujo de matriz de confusion y reporte de clasificacion

In [ ]:
def draw_confusion_matrix(gold_labels, predicted_labels, label_names):
    labels = list(range(len(label_names)))
    cm = confusion_matrix(
        gold_labels,
        predicted_labels,
        labels=labels
    )
    fig, ax = plt.subplots(figsize=(14, 12))  
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm,
        display_labels=label_names
    )
    disp.plot(ax=ax, cmap="Blues", values_format="d")
    plt.title("Confusion Matrix")
    plt.show()

def draw_classification_report(gold_labels, predicted_labels, label_names):
    print(classification_report(
        gold_labels,
        predicted_labels,
        labels=list(range(len(label_names))),
        target_names=label_names
    ))